In [1]:
!pip install crewai langchain_community ollama langchain


In [2]:
import os
from crewai import Agent, Task, Crew
from langchain_ollama import OllamaLLM
from crewai import LLM

In [3]:
# Set environment variables (replace with your actual values)
os.environ["OPENAI_API_KEY"] = "NA"  # If using OpenAI
os.environ["OPENAI_MODEL_NAME"] = 'llama3'
OLLAMA_BASE_URL = os.environ.get("OLLAMA_BASE_URL", "http://localhost:11434")

# Initialize LLM
llama_llm = OllamaLLM(base_url=OLLAMA_BASE_URL, model="llama2")
llama_llm_crew = LLM(
    model="ollama/llama3",
    base_url="http://localhost:11434")


In [4]:
def create_agent(role, goal, backstory):
    return Agent(
        role=role,
        goal=goal,
        llm=llama_llm_crew,
        verbose=True,
        backstory=backstory
    )


In [5]:
# Define agents
research_agent = create_agent(
    role="Researcher",
    goal="Gather comprehensive information on a given topic.",
    backstory="A meticulous researcher with expertise in various fields."
)



In [6]:
writer_agent = create_agent(
    role="Content Writer",
    goal="Create well-structured and engaging content based on research.",
    backstory="An experienced writer skilled in synthesizing complex information."
)



In [7]:
editor_agent = create_agent(
    role="Editor",
    goal="Ensure the document is coherent, accurate, and meets the required length.",
    backstory="A detail-oriented editor with a strong command of language."
)

In [8]:
# Define tasks
def create_task(description, agent, context=None, expected_output=None):
    return Task(
        description=description,
        agent=agent,
        context=context,
        expected_output=expected_output  # Add this line
    )



In [9]:
research_task = create_task(
    description="Research the given topic thoroughly, focusing on the specified sections: Introduction, Science, Applications, Challenges, Future Prospects, Impact on Society, Key Concepts, Real-World Examples, Historical Perspective, and Innovations.",
    agent=research_agent,
    expected_output="A collection of researched information on the topic." # Added expected output
)





In [10]:
writing_task = create_task(
    description="Write a detailed two-page document based on the research provided. Each section should be well-developed and contribute to a cohesive and informative document.",
    agent=writer_agent,
    context=[research_task],
    expected_output="A two-page document draft." # Added expected output
)





In [11]:
editing_task = create_task(
    description="Edit the document for clarity, accuracy, grammar, and style. Ensure the document is approximately two pages in length and covers all required sections effectively.",
    agent=editor_agent,
    context=[writing_task],
    expected_output="A polished and edited two-page document." # Added expected output
)

In [12]:
def generate_document(topic):
    """Generates a two-page document on a given topic."""

    # Update task descriptions with the specific topic
    research_task.description = research_task.description.replace("given topic", f"the topic of {topic}")
    writing_task.description = writing_task.description.replace("research provided", f"research on {topic}")

    crew = Crew(
        agents=[research_agent, writer_agent, editor_agent],
        tasks=[research_task, writing_task, editing_task],
        verbose=True
    )

    result = crew.kickoff(inputs={"topic": topic})
    return result


In [13]:
# Example usage
topic = "Artificial Intelligence"
document = generate_document(topic)

print("Generated Document:")
print(document)

# Optional: Save the document to a file
with open(f"{topic.replace(' ', '_')}_document.txt", "w") as f:
    f.write(str(document))
print(f"Document saved to {topic.replace(' ', '_')}_document.txt")

# Agent: Researcher
## Task: Research the the topic of Artificial Intelligence thoroughly, focusing on the specified sections: Introduction, Science, Applications, Challenges, Future Prospects, Impact on Society, Key Concepts, Real-World Examples, Historical Perspective, and Innovations.


# Agent: Researcher
## Final Answer: 
**Introduction**

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. The term AI was coined in 1956 by computer scientist John McCarthy, but it wasn't until the 1980s and 1990s that significant progress was made in the field. Today, AI is a rapidly growing field with applications in various industries, including healthcare, finance, education, and entertainment.

**Science**

AI is based on mathematical models and algorithms that enable machines to learn from data, make decisions, and improve their performan